# Project 3
**Group 2** 
Ada Matilde Gige (adgi@itu.dk), Albert Schiffer (albsc@itu.dk), Andreas Frederik Flensted Olsen (frao@itu.dk), Timothy Beck (tibe@itu.dk) and Victor Popp Henriksen (vhen@itu.dk)


# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from PIL import Image 
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm


from os import walk #Used for getting all the filenames from a given directory !!!!!MAYBE NOT NECESSARY!!!!!
import glob #For finding specific file types

%run -i ../fyp2021p3_group00_functions.py

# Loading raw datafiles

In [ ]:
#load all images of type .jpg
im_file_list2 = glob.glob("../data/example_image/*.jpg")
#load all maps of type .png
map_file_list2 = glob.glob("../data/example_segmentation/*.png")

#im_files = sorted(im_file_list2, key = uniq)
im_files = sorted(im_file_list2)
len(im_files)
print(im_files[0])

#map_files = sorted(map_file_list2, key = uniq)
map_files = sorted(map_file_list2)
len(map_files)         

# Loading the true file in
true = pd.read_csv('../data/example_ground_truth.csv')
# It is already sorted why it is not sorted as the others



## Loading the images into a list and making an ID dictionary
Taken from: https://stackoverflow.com/questions/33369832/read-multiple-images-on-a-folder-in-opencv-python/33371454

In [ ]:
#Load in all .jpg files from the 'example_image' folder
images = [plt.imread(file) for file in im_files]
#Load in all .png files from the 'example_segmentation' folder
maps = [plt.imread(file) for file in map_files]

# Making a dictionary to find the id from the index
pic_id = dict()

for i in range(150):
    pic_id[i] = true['image_id'][i]
    

# Changing a normal image to luminace image
Making an image as the segmentation masks

In [ ]:
def tolum(image):
    '''A function that takes a pictures filename, the picture must be two colored'''
    image = Image.open(image).convert('L')
    imagenp = np.asarray(image)
    return image , imagenp
    

In [ ]:
image , imagenp = tolum('circle.png')

plt.imshow(imagenp)

# Features

## Asymmetry

In [ ]:
#A function that finds the center of a picture and meassures the assymmetri

def assymmetry(mapID):
    mask = maps[mapID]
    
    borders = np.where(mask == 1) # This will return 2 arrays with the index where the pixels are ones
    up, down, left, right = max(borders[0]), min(borders[0]), min(borders[1]), max(borders[1])
    center = ((left + right)//2, (up+down) //2) # Tuple with the coordinates for the center of the lesion
    
    difference = 0
    
    for i in range(18):
        borders = np.where(mask == 1) # This will return 2 arrays with the index where the pixels are ones
        left, right = min(borders[1]), max(borders[1])
        
        
        radiusdiff = (center[0]-left) - (right-center[0])
        difference += radiusdiff**2
        mask = transform.rotate(mask, 10, center = center)
    return difference/areas[mapID]
        

In [ ]:
asymmetry = pd.read_csv("../data/features/assymmetry.csv", names = ['asymmetry'])

In [ ]:
asymmetry

## Color segmentation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.segmentation import mark_boundaries

In [ ]:
'''This cell block loads in the images as imageobjects and resizes them. Afterwards they are being appended
to a list to store the new image objects'''

images = []
for file in im_files:
    im = Image.open(file)
    im = im.resize((800,500), resample=1) 
    images.append(im)
    

maps = []

for file in map_files:
    ma = Image.open(file)
    ma = ma.resize((800,500), resample=1)
    maps.append(ma)

In [ ]:
def color_segmentation(list_of_images, list_of_corresponding_masks): #lists with resized and resampled images
    '''this function takes the 2 lists as input, the one list with images, and the other with its corresponding masks (both resized)
    For each masked image, it performs a felzensvalb algorithim, to compute the segmentation of the mole. The segmentation is 
    normalized (divided by the totalt number of pixels in the mole), and afterwards put into the dataframe as a feature.
    '''
    segmentation = {}
    
    for i in range(len(list_of_images)): #the file list
        img1 = list_of_images[i]
        print(img1)

        img2 = list_of_corresponding_masks[i]

        img2.paste(img1, (0,0), mask = img2) 

        segments_fz = felzenszwalb(img2, scale=8, sigma=1, min_size=10)

        number_of_segments = len(np.unique(segments_fz))

        normalized_segmentation = round(number_of_segments/(np.sum(list_of_corresponding_masks[i])),7)

        segmentation[i] = normalized_segmentation
    
    return segmentation

In [ ]:
#call the function
#felz = color_segmentation(images, maps) 

''' Exported as a csv file to avoid future run time
'''

In [ ]:
'''To see what is going on inside the loop, is a cutout for only 1 picture in this block of code

'''

img1 = images[0]
img2 = maps[0]
img2.paste(img1, (0,0), mask = img2) 
segments_fz = felzenszwalb(img2, scale=8, sigma=1, min_size=10)
number_of_segments = len(np.unique(segments_fz))
normalized_segmentation = round(number_of_segments/(np.sum(maps[0])),7)
plt.imshow(mark_boundaries(img2, segments_fz))
print('pixels in mask divided by area of mask = ', normalized_segmentation)
#segmenation[images[i]] = normalized_segmentation

In [ ]:

np.savetxt("../data/features/segmentation.csv", list(felz.values()), delimiter=",")

## Border
### Area and perimiter

In [ ]:
# Finding the area and perimeter of all images. 

areas = []
peris = []
for i in maps:
    area, perimiter = measure_area_perimeter(i)
    areas.append(areas)
    peris.append(perimiter)
    
# Adding them to a file, so they don't have to be calculated each time

#np.savetxt("../data/features/perimeter.csv", peris, delimiter=",")
#np.savetxt("../data/features/area.csv", areas, delimiter=",")

In [ ]:
areas = np.genfromtxt('../data/features/areas.csv', delimiter=',')
perimeter = np.genfromtxt('../data/features/perimeter.csv', delimiter=',')

### Area and perimeter scatterplot

In [ ]:
def scatterplot(area, perimeter):
    # Define the size of the figure
    fig = plt.figure(figsize=(4, 3))
    # Set axes, that you want to work with:
    axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
    # actually plot the data:
    axes.scatter(area, perimeter, label = 'Data', edgecolors='white')
    axes.set_title('Area and perimeter')
    axes.set_xlabel('Area')
    axes.set_ylabel('Perimeter')
    #axes.legend(loc='upper left'); axes.set_xlabel(data); axes.set_ylabel(data); axes.set_title('Scatter plot');
    return None

In [ ]:
scatterplot(areas, perimeter)

## Perimeter divided by area

In [ ]:
#making the perimiter divided by area feature

pa = []
for i in range(150): 
    pa.append(perimeter[i]/areas[i])
    
# Making a file with the perimeter divided by area feature
#np.savetxt("../data/features/perimeterdivarea.csv", areas, delimiter=",")


In [ ]:
pa = np.genfromtxt('../data/features/perimeterdivarea.csv', delimiter=',')

### Making a histogram for the perimeter/area

In [ ]:
sns.histplot(pa, bins = 20)

# All features in one dataframe

In [ ]:
true['unhealthy'] = 0

for i in range(150):
    if true['melanoma'][i] == 1 or true['seborrheic_keratosis'][i] == 1:
        true['unhealthy'][i] = 1

areas = pd.read_csv("../data/features/areas.csv", names = ['area'])
perimeter = pd.read_csv("../data/features/perimeter.csv", names = ['perimeter'])
pa = pd.read_csv('../data/features/perimeterdivarea.csv', names = ['peri/area'])
asymmetry = pd.read_csv("../data/features/assymmetry.csv", names = ['asymmetry'])
segmentation = pd.read_csv("../data/features/segmentation.csv", names = ['color segmentation/area'])

true['asymmetry'] = asymmetry
true['area'] = areas
true['perimeter'] = perimeter
true['peri/area'] = pa
true['color segmentation/area'] = segmentation
allfeatures = true
        
allfeatures

In [ ]:
# Plotting the features

onlyfeatures = allfeatures[['area','perimeter', 'asymmetry', 'peri/area', 'color segmentation/area']]
sns.boxplot(data=onlyfeatures, width=0.5)

In [ ]:
# Scaling the features

#Fit scaler on our data
scaler = preprocessing.StandardScaler().fit(onlyfeatures)

#Apply to data itself
normfeatures = scaler.transform(onlyfeatures)

print(normfeatures.mean()) #small number close to 0, round of error
print(normfeatures.var())  #equal to 1 
sns.boxplot(data=normfeatures, width=0.5,fliersize=5) #we see both negative and positive values, since the mean is 0


In [ ]:
normfeatures = pd.DataFrame(normfeatures, columns = ['area','perimeter', 'asymmetry', 'peri/area', 'color segmentation/area'])

# Look at values per class
normfeatures['unhealthy'] = allfeatures['unhealthy']



sns.pairplot(normfeatures, hue="unhealthy", size=3,diag_kind="hist")

In [ ]:
# Split the data before feature selection
from sklearn.model_selection import train_test_split

# Some noisy data not correlated
noise = np.random.RandomState(42).uniform(0, 0.1, size=(normfeatures.shape[0], 20))

# Add the noisy data to the informative features
X = np.hstack((normfeatures[['area', 'perimeter','asymmetry', 'peri/area', 'color segmentation/area']], noise))
y = normfeatures['unhealthy']

# Split dataset to select feature and evaluate the classifier
X_dev, X_test, y_dev, y_test = train_test_split(
        X, y, stratify=y, random_state=0)

X_train, X_val, y_train, y_val = train_test_split(
        X_dev, y_dev, stratify=y_dev)

In [ ]:
# Univariate feature selection with mutual information for feature scoring
selector = SelectKBest(mutual_info_classif, k=4)
selector.fit(X_train, y_train)

scores = selector.scores_


In [ ]:
# Select features that had good scores on training set
X_train1 = X_train[:, [0,3]]
X_train2 = selector.transform(X_train)

# Train a classifier
knn1 = KNeighborsClassifier(n_neighbors=1) # other hyperparameters possible
knn1trained = knn1.fit(X_train2, y_train)

knn2 = KNeighborsClassifier(n_neighbors=3)
knn2trained = knn2.fit(X_train2, y_train)

tree1 = DecisionTreeClassifier() # various hyperparameters
tree1trained = tree1.fit(X_train2, y_train)

svm1 = svm.SVC()
svmtrained = svm1.fit(X_train2, y_train)

In [ ]:
#Select the same features as before
X_val1 = X_val[:, [0,3]]
X_val2 = selector.transform(X_val)

y_val_knn1 = knn1trained.predict(X_val2)
y_val_knn2 = knn2trained.predict(X_val2)
y_val_svm1 = svmtrained.predict(X_val2)
y_val_tree = tree1trained.predict(X_val2)

# Simple accuracy
print(np.sum(y_val_knn1 == y_val) / np.size(y_val) * 100)
print(np.sum(y_val_knn2 == y_val) / np.size(y_val) * 100)
print(np.sum(y_val_svm1 == y_val) / np.size(y_val) * 100)
print(np.sum(y_val_tree == y_val) / np.size(y_val) * 100)